In [ ]:
import tqdm, re, pandas as pd, random, pprint, seaborn as sns, matplotlib.pyplot as plt, pickle as pkl
from mpl_toolkits.axes_grid.inset_locator import inset_axes

In [ ]:
def get_memory():
    status= open('/proc/self/status').read()
    memuse= int(re.findall('VmRSS:\s*(\d+)\skB', status)[0]) / 1024 ** 2
    print('Resident memory: %.2f GB' % memuse)

In [ ]:
get_memory()

In [ ]:
def loadPickle(name):
    return pd.read_pickle(f'../data/{name}')
def loadTxt(name):
    return [i.strip() for i in open(f'../../data/{name}', 'r').readlines()]
def dumpPickle(df, name):
    df.to_pickle(f'../data/{name}')

In [ ]:
Normal_Data= {'user_creation': set()}
with open(f'../../data/all_tweets_normal_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                user_creation= i['user']['created_at']
                Normal_Data['user_creation'].add(user_creation)
    except EOFError:
        pass
Normal_Data['user_creation']= list(Normal_Data['user_creation'])

In [ ]:
restoredUsersList= set()
for i in loadTxt('unsuspended_8_Oct.txt'):
    restoredUsersList.add(i)

In [ ]:
Suspended_Data= {'user_creation': set()}
Restored_Data= {'user_creation': set()}
with open(f'../../data/all_tweets_suspended_jan3_2021.pickle', 'rb') as f:
    try:
        while True:
            temp= pkl.load(f)
            for i in tqdm.tqdm(temp):
                user_creation= i['user']['created_at']
                user_id= i['user']['id_str']
                if user_id in restoredUsersList:
                    Restored_Data['user_creation'].add(user_creation)
                else:
                    Suspended_Data['user_creation'].add(user_creation)
    except EOFError:
        pass
Suspended_Data['user_creation']= list(Suspended_Data['user_creation'])
Restored_Data['user_creation']= list(Restored_Data['user_creation'])

In [ ]:
Normal_Data= pd.DataFrame.from_dict(Normal_Data)
Suspended_Data= pd.DataFrame.from_dict(Suspended_Data)
Restored_Data= pd.DataFrame.from_dict(Restored_Data)

In [ ]:
Normal_Data['user_creation']= pd.to_datetime(Normal_Data['user_creation'])
Suspended_Data['user_creation']= pd.to_datetime(Suspended_Data['user_creation'])
Restored_Data['user_creation']= pd.to_datetime(Restored_Data['user_creation'])

In [ ]:
Normal_Data['user_creation']= Normal_Data['user_creation'][Normal_Data['user_creation'] > '2006-01-01']
Suspended_Data['user_creation']= Suspended_Data['user_creation'][Suspended_Data['user_creation'] > '2006-01-01']
Restored_Data['user_creation']= Restored_Data['user_creation'][Restored_Data['user_creation'] > '2006-01-01']

In [ ]:
def plot_creation_date(dt_col, label, x1= 1, x2= 1, y1= 1, y2= 1):
    ser= pd.Series(1, index= dt_col).resample('M').agg('count')
    ser.index= ser.index.to_period('M')
    tot_counts= ser.sum()
    ser/= tot_counts
    ax= ser.plot(c= '0.5')
    ax.set_ylim(0, 0.3)
    ax.set_xlabel("Creation Date", fontsize = 16) 
    ax.set_ylabel("Fraction of Accouts", fontsize = 16)
    axins= inset_axes(ax, 2, 1.5, loc=2, bbox_to_anchor= (0.2, 0.85), bbox_transform= ax.figure.transFigure)
    axins= ser.plot(marker= 'o', c= '0.5')
    axins.set(xlabel= "")
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)

In [ ]:
plot_creation_date(Normal_Data['user_creation'], "Normal Accounts", x1= '2019-01', x2= '2019-05', y1= 0, y2= 0.3)

In [ ]:
plot_creation_date(Suspended_Data['user_creation'], "Suspended Accounts", x1= '2019-01', x2= '2019-05', y1= 0, y2= 0.3)

In [ ]:
plot_creation_date(Restored_Data['user_creation'], "Restored Accounts", x1= '2019-01', x2= '2019-05', y1= 0, y2= 0.3)